# Preparação do ambiente

## Bibliotecas

In [33]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import string

from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from os import path as osp
from tqdm.auto import tqdm
from unicodedata import normalize
from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer

## Constantes e funções auxiliares

In [2]:
FILE_ITEMS = '../datasets/label_issues/20240423/results/items.parquet'
FILE_SAMPLES_RESULTS = '../datasets/label_issues/20240423/results/samples_results.parquet'

In [58]:
def clean_text(doc, return_tokens=False):

    stop_words = stopwords.words('portuguese')
    stop_words.extend(stopwords.words('english'))
    stop_words.extend(list(string.punctuation))
    
    # stopwords específicas do domínio
    # stop_words.extend(['cm', 'feature', 'features', 'informações', 'itens', 'leve', 'list', 'nulo', 'package', 'pacote', 'pacotes', 'recurso', 'tamanho', 'ver'])
    # remover da lista de stopwords a palavra sem para formar o bigrama "sem fio", relevante para o domínio
    # primeira rodada de classificação demonstrou que não foi uma boa escolha
    # stop_words.remove('sem')
    
    doc = doc.lower()
    doc = normalize('NFKD', doc).encode('ASCII', 'ignore').decode('ASCII')
    
    # nltk tokenizer
    tokens = [token for token in word_tokenize(doc, language='portuguese') if token not in stop_words]
    

    if return_tokens:
        return tokens
    else:
        return ' '.join(tokens)

# Carga e prepação dos dados

In [62]:
df_items = pd.read_parquet(FILE_ITEMS)
df_items['doc'] = df_items['nome'].map(clean_text)
df_items['tokens'] = df_items['nome'].apply(lambda doc: clean_text(doc, return_tokens=True))

columns_to_keep = ['id', 'nome', 'doc', 'tokens', 
                   'titulo_bem_elaborado', 'titulo_identifica_produto',
                   'passivel_homologacao', 'transmissor_rf', 'tipo_produto']

df_items = df_items[columns_to_keep]
df_items

,id,nome,doc,tokens,titulo_bem_elaborado,titulo_identifica_produto,passivel_homologacao,transmissor_rf,tipo_produto
0,126504,Controle Remoto Tv Samsung Smart,controle remoto tv samsung smart,"[controle, remoto, tv, samsung, smart]",1,1,1,1,Transceptor de radiofrequência (VHF/UHF)
1,126505,Smartphone Redmi 9t Xiaomi Com Película E Capa...,smartphone redmi 9t xiaomi pelicula capa seminovo,"[smartphone, redmi, 9t, xiaomi, pelicula, capa...",1,1,1,1,Smartphones
2,126506,Drone DJI Mini 3 Pro DJI RC-N1 (Sem tela) Fly ...,drone dji mini 3 pro dji rc-n1 tela fly kit pl...,"[drone, dji, mini, 3, pro, dji, rc-n1, tela, f...",1,1,1,1,Drone
3,126507,Smartphone Samsumg J5 Prime,smartphone samsumg j5 prime,"[smartphone, samsumg, j5, prime]",1,1,1,1,Smartphones
4,126508,"Tablet PC, 12 GB RAM 256 GB ROM US Plug 100-24...",tablet pc 12 gb ram 256 gb rom us plug 100-240...,"[tablet, pc, 12, gb, ram, 256, gb, rom, us, pl...",1,1,1,1,Transceptor de radiação restrita (2.4GHz/5.8GHz)
...,...,...,...,...,...,...,...,...,...
4347,125715,CAIXA ACUSTICA PCX6500,caixa acustica pcx6500,"[caixa, acustica, pcx6500]",0,1,1,1,Transceptor de radiação restrita (2.4GHz/5.8GHz)
4348,125716,Conector de cauda de celular 10 pcs carregando...,conector cauda celular 10 pcs carregando conec...,"[conector, cauda, celular, 10, pcs, carregando...",1,1,0,0,Outro tipo/categoria (não listada acima)
4349,125717,Avantree Medley Clear - Fones de ouvido sem fi...,avantree medley clear fones ouvido fio ouvir t...,"[avantree, medley, clear, fones, ouvido, fio, ...",1,1,1,1,Transceptor de radiação restrita (2.4GHz/5.8GHz)
4350,125718,"Hilitand Adaptador sem fio, placa de rede sem ...",hilitand adaptador fio placa rede fio adaptado...,"[hilitand, adaptador, fio, placa, rede, fio, a...",1,1,1,1,Transceptor de radiação restrita (2.4GHz/5.8GHz)


# Análise

## Frequência de termos e de documentos

In [63]:
docs = df_items['tokens']
tokens_tf = Counter()
tokens_df = Counter()
for doc in tqdm(docs):
    unique_tokens = set(doc)
    tokens_tf.update(doc)
    tokens_df.update(unique_tokens)

  0%|          | 0/4352 [00:00<?, ?it/s]